In [1]:
import sys 
sys.path.append('../..') 

In [2]:
import numpy as np
import pandas as pd

In [3]:
import time

In [4]:
import pulp

In [5]:
from src.modeling import *

In [6]:
from src.graphs import * 

In [7]:
def of_net_income(_cost, _vars, _crops_year):
    return pulp.lpSum([getattr(_cost, _crops_year[i]) * _vars[i] for i in _crops_year])

def of_ua_total(_vars, _crops_year):
    return pulp.lpSum([_vars[i] for i in _crops_year])

def of_crop_diversity(_vars, _crops_year):
    return pulp.lpSum([_vars[i] for i in _crops_year])

In [8]:
DATA_PATH = '../../data/al_rasheed'
RESULTS_PATH = '../../results/data/al_rasheed/model_1/econst'

In [9]:
variables = pd.read_csv(f'{DATA_PATH}/variables.csv', sep=';')
income = pd.read_csv(f'{DATA_PATH}/net_income.csv', sep=';')
constraints = pd.read_csv(f'{DATA_PATH}/constraints.csv', sep=';')
rotations = pd.read_csv(f'{DATA_PATH}/rotations.csv', sep=';').fillna(0)

In [10]:
#YEARS = 1

In [11]:
crops = variables.variable.to_list()

crops_year_integer = dict(zip([f"y{i:03}_int_{j}" for i in range(1,YEARS+1) for j in crops], YEARS*crops))

crops_year_binary = dict(zip([f"y{i:03}_bin_{j}" for i in range(1,YEARS+1) for j in crops], YEARS*crops))

In [12]:
variables.set_index(keys='variable', inplace=True)

integer_vars = create_integer_vars(crops_year_integer, variables)

binary_vars = create_binary_vars(crops_year_binary, variables)

In [13]:
ofs = [of_net_income(income.loc[0,:], integer_vars, crops_year_integer),
       of_ua_total(integer_vars, crops_year_integer),
       of_crop_diversity(binary_vars, crops_year_binary)]

In [14]:
min_max_ofs = []
for of in ofs:
    vl = []
    for opt in [pulp.LpMinimize, pulp.LpMaximize]:
        model = pulp.LpProblem("Modelo_1", opt)
        model += of
        const_idx = add_constraints(model, integer_vars, constraints, crops_year_integer, len(crops), YEARS, const_id = 0)
        const_idx = add_binary_correlation(model, binary_vars, integer_vars, 113, const_id = const_idx)
        const_idx = add_crop_rotation_constraints(model, integer_vars, crops_year_integer, rotations, YEARS, crops, 'y{:03d}_int_{}', const_idx)

        solution = model.solve(pulp.GLPK())

        vl.append(pulp.value(model.objective))
    
    min_max_ofs.append(vl)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/ef56c49b6a4e4a7c941ea279168fad86-pulp.lp -o /tmp/ef56c49b6a4e4a7c941ea279168fad86-pulp.sol
Reading problem data from '/tmp/ef56c49b6a4e4a7c941ea279168fad86-pulp.lp'...
43 rows, 24 columns, 212 non-zeros
24 integer variables, 12 of which are binary
157 lines were read
GLPK Integer Optimizer 5.0
43 rows, 24 columns, 212 non-zeros
24 integer variables, 12 of which are binary
Preprocessing...
12 constraint coefficient(s) were reduced
31 rows, 24 columns, 118 non-zeros
24 integer variables, 12 of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.260e+03  ratio =  2.260e+03
GM: min|aij| =  4.457e-01  max|aij| =  2.244e+00  ratio =  5.033e+00
EQ: min|aij| =  2.114e-01  max|aij| =  1.000e+00  ratio =  4.731e+00
2N: min|aij| =  1.104e-01  max|aij| =  1.562e+00  ratio =  1.416e+01
Constructing initial basis...
Size of triangular part is 31
Solving LP relaxation...
GLPK Simplex Optimizer 5.

In [15]:
f_epsilon = lambda min_max, n_elements: np.linspace(min_max[0], min_max[1], n_elements, dtype=int)

In [16]:
solution_lst = []

for eps_1 in f_epsilon(min_max_ofs[1], 23):
    for eps_2 in f_epsilon(min_max_ofs[2], 1000):

        start_time = time.time()

        model = pulp.LpProblem("First_Model", pulp.LpMaximize)

        epsilon_constraint(model, ofs, 0, [eps_1, eps_2]) 

        const_idx = add_constraints(model, integer_vars, constraints, crops_year_integer, len(crops), YEARS, const_id = 0)
        const_idx = add_binary_correlation(model, binary_vars, integer_vars, 113, const_id = const_idx)
        const_idx = add_crop_rotation_constraints(model, integer_vars, crops_year_integer, rotations, YEARS, crops, 'y{:03d}_int_{}', const_idx)

        solution = model.solve(pulp.GLPK())

        end_time = time.time()

        integer_var_values = {key: pulp.value(value) for key, value in integer_vars.items()}
        binary_var_values = {key: pulp.value(value) for key, value in binary_vars.items()}

        ney_incone_ofs_values = sum_cost_of_values(income.loc[0,:], integer_var_values, crops_year_integer, 'net_income', 0, 4)
        us_ofs_values = sum_of_var_values(integer_var_values, 'ua', 0, 4)
        crop_diversity_ofs_values = sum_of_var_values(binary_var_values, 'crop_diversity', 0, 4)

        solution_lst.append([
                    end_time - start_time,
                    eps_1,
                    eps_2,
                    str(pulp.LpStatus[solution])] +
                    list(integer_var_values.values())+
                    list(binary_var_values.values())+
                    [pulp.value(model.objective)] +
                    list(ney_incone_ofs_values.values())+
                    list(us_ofs_values.values())+
                    list(crop_diversity_ofs_values.values())
                    )

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/55fdac5a0dfa4d4785b41267ef5b8f1c-pulp.lp -o /tmp/55fdac5a0dfa4d4785b41267ef5b8f1c-pulp.sol
Reading problem data from '/tmp/55fdac5a0dfa4d4785b41267ef5b8f1c-pulp.lp'...
45 rows, 24 columns, 236 non-zeros
24 integer variables, 12 of which are binary
165 lines were read
GLPK Integer Optimizer 5.0
45 rows, 24 columns, 236 non-zeros
24 integer variables, 12 of which are binary
Preprocessing...
12 constraint coefficient(s) were reduced
32 rows, 24 columns, 130 non-zeros
24 integer variables, 12 of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.260e+03  ratio =  2.260e+03
GM: min|aij| =  4.581e-01  max|aij| =  2.183e+00  ratio =  4.766e+00
EQ: min|aij| =  2.160e-01  max|aij| =  1.000e+00  ratio =  4.629e+00
2N: min|aij| =  1.104e-01  max|aij| =  1.504e+00  ratio =  1.363e+01
Constructing initial basis...
Size of triangular part is 32
Solving LP relaxation...
GLPK Simplex Optimizer 5.

KeyboardInterrupt: 

In [17]:
columns = ["time", "epsilon_1", "epsilon_2", "status"] +\
         list(integer_var_values.keys()) + \
         list(binary_var_values.keys()) + \
         ['OF_Optimized'] + \
         list(ney_incone_ofs_values.keys()) + \
         list(us_ofs_values.keys()) + \
         list(crop_diversity_ofs_values.keys())

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/e0eb9226ea97460ab783662295065097-pulp.lp -o /tmp/e0eb9226ea97460ab783662295065097-pulp.sol
Reading problem data from '/tmp/e0eb9226ea97460ab783662295065097-pulp.lp'...
45 rows, 24 columns, 236 non-zeros
24 integer variables, 12 of which are binary
165 lines were read
GLPK Integer Optimizer 5.0
45 rows, 24 columns, 236 non-zeros
24 integer variables, 12 of which are binary
Preprocessing...
1 hidden packing inequaliti(es) were detected
12 constraint coefficient(s) were reduced
33 rows, 24 columns, 142 non-zeros
24 integer variables, 12 of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.260e+03  ratio =  2.260e+03
GM: min|aij| =  4.581e-01  max|aij| =  2.183e+00  ratio =  4.766e+00
EQ: min|aij| =  2.160e-01  max|aij| =  1.000e+00  ratio =  4.629e+00
2N: min|aij| =  1.250e-01  max|aij| =  1.504e+00  ratio =  1.203e+01
Constructing initial basis...
Size of triangular part is 33
Solv

In [18]:
results = pd.DataFrame(solution_lst, columns=columns)

In [19]:
results.to_csv(f'{RESULTS_PATH}/EpsilonConstraint_y{YEARS:03}.csv', sep=';', index=False)